In [1]:
import mapboxfetcher
from mapboxfetcher import MapBoxFetcher
import utils
import matplotlib.pyplot as plt
import numpy as np
import json
import imageio
from importlib import reload
from pylab import rcParams
import pickle
from PIL import Image
from random import shuffle
rcParams['figure.figsize'] = 10, 10
%matplotlib inline

reload(mapboxfetcher)
reload(utils)

<module 'utils' from 'C:\\Users\\serge\\Documents\\EPFL\\Personal Interaction Studio\\Code\\utils.py'>

In [21]:
from functools import partial
import pyproj
from shapely.ops import transform
import shapely.ops as ops

city = json.load(open('GeoJSON/lakeleman.geojson'))
# city_polygon = utils.polygon_union(city['features'])
city_polygon = city['features'][0]
city_bounds = city_polygon.bounds

# transform_func = partial(
#     pyproj.transform,
#     pyproj.Proj(init='EPSG:4326'),
#     pyproj.Proj(
#         proj='aea',
#         lat1=city_polygon.bounds[1],
#         lat2=city_polygon.bounds[3],
#         lon1=city_polygon.bounds[0],
#         lon2=city_polygon.bounds[2]
#     )
# )

# transformed_polygon = ops.transform(transform_func, city_polygon)

# print(transformed_polygon.area)

AttributeError: 'dict' object has no attribute 'bounds'

In [26]:


# all_style_chunks = []
# all_sat_chunks = []

all_sat_imgs = []
all_style_imgs = []

fetcher = MapBoxFetcher()
# fetcher  = MapBoxFetcher(style_id='cjfuwjonm2r8p2sny75hyaegi')
zoom = 16 # https://wiki.openstreetmap.org/wiki/Zoom_levels 18 = 0.596m/pixel
lat = city_bounds[3]
lon = city_bounds[0]
idx = 0
while lat >= city_bounds[1]:
    idx += 1
    while lon <= city_bounds[2]:
        if idx != 10:
            lon = utils.num2deg(utils.deg2num(lat, lon, zoom)[0] + 1, utils.deg2num(lat, lon, zoom)[1], zoom)[1]
            continue
        numx, numy = utils.deg2num(lat, lon, zoom)
        style_img = fetcher.style(lat, lon, zoom, hq=True)
        sat_img = fetcher.satellite(lat, lon, zoom, hq=True)
        
        all_sat_imgs.append(sat_img)
        all_style_imgs.append(style_img)
        
        ### CODE
#         style_chunks = utils.split_image(style_img.bw_array(), chunk_size=2)
#         style_chunks = utils.split_image(style_img.array(), chunk_size=512)
#         sat_chunks = utils.split_image(sat_img.array(), chunk_size=2)
#         all_style_chunks.extend(style_chunks)
#         all_sat_chunks.extend(sat_chunks)
#         plt.figure()
#         plt.imshow(sat_chunks[0])
#         plt.figure()
#         plt.imshow(style_chunks[0], cmap='gray')
        ### END CODE
        
#         lon = 999999; lat = -1
        lon = utils.num2deg(utils.deg2num(lat, lon, zoom)[0] + 1, utils.deg2num(lat, lon, zoom)[1], zoom)[1]
    lon = city_bounds[0]
    lat = utils.num2deg(utils.deg2num(lat, lon, zoom)[0], utils.deg2num(lat, lon, zoom)[1] + 1, zoom)[0]
    if idx == 10:
        break

In [3]:
all_style_imgs_arrays = []
all_sat_imgs_arrays = []
for style_img in all_style_imgs:
    all_style_imgs_arrays.append(style_img.bw_array())
for sat_img in all_sat_imgs:
    all_sat_imgs_arrays.append(sat_img.array())
pickle.dump({'style': all_style_imgs_arrays, 'sat': all_sat_imgs_arrays}, open('images-lausanne-zoom-{}.p'.format(zoom), 'wb'))

In [27]:
saved = 0
for sat_img in all_sat_imgs:
#     if np.count_nonzero(all_style_imgs[i].bw_array()) > 0:
    sat_img.save('TOLABELWATER/{}.jpeg'.format(saved))
    saved += 1
    if saved >= int(len(all_sat_imgs) * 1):
        break

In [2]:
data = pickle.load(open('images-lausanne-zoom-16.p', 'rb'))
to_save_images = data['sat']
img = Image.fromarray(to_save_images[651])
img.save('TOLABEL16/{}.jpeg'.format(70))
# shuffle(data['sat'])
# saved = 0
# for sat_img in to_save_images:
#     img = Image.fromarray(sat_img)
#     img.save('TOLABEL16/{}.jpeg'.format(saved))
#     saved += 1
#     if saved >= int(len(to_save_images) * 0.1):
#         break